In [ ]:
# 导入库文件，方便后续调用

from sklearn.feature_extraction import DictVectorizer   #标称型特征向量化
from sklearn.compose import ColumnTransformer   #特征列处理
from sklearn.pipeline import Pipeline   #数据转换、模型多步骤处理
from sklearn.impute import SimpleImputer   #填充缺失值
from sklearn.preprocessing import StandardScaler, OneHotEncoder   #标准化、向量化
from sklearn.model_selection import train_test_split, GridSearchCV   #划分训练集、网络检索（参数调优）
from sklearn.tree import DecisionTreeClassifier   #决策树
from sklearn.ensemble import RandomForestClassifier   #随机森林
from sklearn.linear_model import LogisticRegression, SGDClassifier #线性回归
from sklearn.neighbors import KNeighborsClassifier   #KNN
from sklearn.naive_bayes import GaussianNB #高斯GB
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap
import plotly.express as px
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["font.serif"] = ["SimHei"]

# Data understanding and cleaning

In [ ]:
# 数据加载
data = pd.read_csv('../input/hotel-booking-demand/hotel_bookings.csv')

In [ ]:
# 查看数据基本信息
data.info()
print('---------------------------------------------------------------------''\n'
      'hotel_bookings数据集中，共有{}个数据样本，每个样本有{}个数据字段.'.format(*data.shape))

In [ ]:
# 查看数据前五行
data.head()

In [ ]:
# 查看数据描述
data.describe()

In [ ]:
# 统计缺失值
data.isnull().sum()

In [ ]:
#统计缺失率
data.isnull().sum()/data.shape[0]

可见，共有四列出现缺失值，分别作出如下处理：

* children和country列由于缺失数目较小，采用对应列众数进行填充；
* agent缺失16340个，缺失率为13.6%，缺失数量较大，但agent表示预订的旅行社，且缺失率小于20%，保留，并用0填充，表示没有旅行社ID；
* company缺失112593个，缺失率为94.3%>80%,不具备信息价值有效性，所以直接删除

In [ ]:
# 缺失值处理

#复刻样本数据，不对数据源做处理
df = data.copy()

df.children.fillna(df.children.mode()[0],inplace=True)
df.country.fillna(df.country.mode()[0],inplace=True)
df.agent.fillna(0, inplace=True)
df.drop('company',inplace=True, axis=1)

存在四种异常
* 存在入住总人数为0和总入住天数为0的异常数据，需要对这些数据进行筛选和清理。
* meal里总计有5种餐型，其中Undefined / SC –无餐套餐为一类，需统一替换为SC类
* 小孩的入住量、旅行社代理入住的量均不能是浮点数，需要统一规划为整型数据。
* 酒店平均每日收费有一个大于5000的离群值，会严重影响描述性统计，需删除离群值

In [ ]:
# 异常值处理

# 入住人数为0
zero_guest=df[df[['adults', 'children', 'babies']].sum(axis=1)==0]
df.drop(zero_guest.index, inplace=True)

# 入住天数为0
zero_days = df[df[['stays_in_weekend_nights','stays_in_week_nights']].sum(axis=1) == 0]
df.drop(zero_days.index, inplace=True)

# 餐食类型Undefined/SC合并
df.meal.replace("Undefined", "SC", inplace=True)

# # children、agent字段不可能为浮点数，需修改数据类型
df.children = df.children.astype(int)
df.agent = df.agent.astype(int)

#核实adr变量的离群值情况
sns.boxplot(x=df['adr'])

ax = sns.boxplot(x=df['adr'])

#删除离群值
df = df[df["adr"]<5000]

In [ ]:
#要计算实际的订单数就要去除那些被取消的订单
#为了好理解，数据清洗后，将两家酒店的数据分开 
#从full_data_cln中分出两个酒店的未取消订单数据rh和ch

rh = df.loc[(df["hotel"] == "Resort Hotel") & (df["is_canceled"] == 0)]
ch = df.loc[(df["hotel"] == "City Hotel") & (df["is_canceled"] == 0)]
df['hotel'].value_counts()

In [ ]:
# 方便后续分析，增加两个新列

#增加一列总住宿晚数stays_nights_total
nums_stays = df.stays_in_weekend_nights + df.stays_in_week_nights
df.insert(9,"stays_nights_total",nums_stays)

#增加一列住宿人数number_of_people
nums_peoples = df.adults + df.children + df.babies
df.insert(12,"number_of_people",nums_peoples)
df.info()

数据清洗完成，清洗完的数据集大小变为118564×33

# Data analysis and visualization

## 客房信息分析

In [ ]:
# 查看酒店类型与取消预定的关系

sns.countplot(x=df['hotel'], hue=df['is_canceled'])
plt.show()

* 城市酒店订单量明显超过度假酒店，但同时预订取消的可能性也远远高于度假酒店。

In [ ]:
# 查看房间类型与取消预订的关系
index = 1
for room_type in ['reserved_room_type', 'assigned_room_type']:
    # plt.figure(figsize=(6,8))
    ax1 = plt.subplot(2, 1, index)
    index += 1
    ax2 = ax1.twinx()
    ax1.bar(
        df.groupby(room_type).size().index,
        df.groupby(room_type).size())
    ax1.set_xlabel(room_type)
    ax1.set_ylabel('Number')
    ax2.plot(
        df.groupby(room_type)['is_canceled'].mean(),'ro-')
    ax2.set_ylabel('Cancellation rate')
    plt.show()

* 订单预定和分配的房间类型多数集中在A/D/E/F四类，其中A类房型取消率高出其余三类约7-8个百分点，值得关注。

In [ ]:
# 房间类型变更对取消预定的影响
df['room_changed']=df['reserved_room_type']!=df['assigned_room_type']
sns.countplot(x=df['room_changed'],hue=df['is_canceled'])

* 房型变更过的客户取消预定的概率远远小于未变更过的客户，猜测原因可能为：

客户对于房型不满意，但为保证正常入住，不取消预定而选择更改房型，故取消概率低

## 客户信息分析

In [ ]:
# 查看预定人数与取消预定的关系
plt.figure(figsize=(12, 6))
index = 0
for people in ['adults', 'children', 'babies']:
    index += 1
    plt.subplot(2, 3, index)
    plt.plot(df.groupby(people)['is_canceled'].mean(),
             'ro-',
             ms=4)
    plt.title(people, fontsize=15)

    plt.subplot(2, 3, index + 3)
    people_stats = df[people].value_counts()
    sns.barplot(x=people_stats.index, y=people_stats.values)
plt.tight_layout()
plt.show()

1）多数预定订单没有儿童和婴儿入住，其中单人入住和双人入住是主要的预定人数模式；

2）有婴儿入住时预定取消率大幅下降；

In [ ]:
# 入住人数模式分析

# 单人
single = (df.adults == 1) & (df.children == 0) & (df.babies == 0)
# 双人
couple = (df.adults == 2) & (df.children == 0) & (df.babies == 0)
# 家庭
family = (df.adults >= 2) & (df.children > 0) | (df.babies > 0)

df['people_mode'] = single.astype(int) + couple.astype(int) * 2 + family.astype(int) * 3
plt.figure(figsize=(10,6))
index=1
for hotel_kind in ['City Hotel','Resort Hotel']:
    plt.subplot(1,2,index)
    index+=1
    sns.countplot(x=df['people_mode'],
              hue=df['is_canceled'],
              data=data[data.hotel == hotel_kind])
    plt.xticks([0, 1, 2, 3], ['Others', 'Single', 'Couple', 'Family'])
    plt.title(hotel_kind)
plt.tight_layout()
plt.show()

* 对于城市酒店，取消预定概率：双人>>单人≈家庭，应注意双人入住客户的高取消率现象，改善酒店对于双人入住客户的配套服务以降低取消率。

* 对于度假酒店，取消预订概率：家庭>双人>单人，酒店可适当针对家庭客户提供相应的优惠折扣，提高家庭客户入住率。

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.pie(df[df['is_canceled'] == 1].meal.value_counts(),
        labels=df[df['is_canceled'] == 1].meal.value_counts().index,
        autopct="%.2f%%")
plt.legend(loc=1)
plt.title('Canceled')
plt.subplot(122)
plt.pie(df[df['is_canceled'] == 0].meal.value_counts(),
        labels=df[df['is_canceled'] == 0].meal.value_counts().index,
        autopct="%.2f%%")
plt.legend(loc=1)
plt.title('Uncanceled')

* 客人特别喜爱BB类型餐饮，其次是HB和SC，建议可以BB品类加购HB和SC的活动，促进客人消费
* 无论是否取消预订，餐食类型之间差异不大。

In [ ]:
country_df = pd.DataFrame(df.loc[df["is_canceled"] == 0]["country"].value_counts())


#得出不同国家的顾客人数
country_df = pd.DataFrame(df.loc[df["is_canceled"] == 0]["country"].value_counts())
country_df.rename(columns={"country": "Number of Guests"}, inplace=True)
total_guests = country_df["Number of Guests"].sum()
country_df["Guests in %"] = round(country_df["Number of Guests"] / total_guests * 100, 2)
country_df["country"] = country_df.index

# pie plot 制作饼图
fig = px.pie(country_df,
             values="Number of Guests",
             names="country",
             title="Home country of guests",
             template="seaborn")
fig.update_traces(textposition="inside", textinfo="value+percent+label")
fig.show()

In [ ]:
# show on map  制作地图
guest_map = px.choropleth(country_df,
                    locations=country_df.index,
                    color=country_df["Guests in %"], 
                    hover_name=country_df.index, 
                    color_continuous_scale=px.colors.sequential.Plasma,
                    title="Home country of guests")
guest_map.show()

* 由图可知，客户主要来自葡萄牙，英国，法国，西班牙等欧洲国家，不同国家之间预定取消率的差距非常显著，取消率较高的国家有葡萄牙、意大利、巴西、中国、俄罗斯，以发展中国家为主。

In [ ]:
sns.countplot(x=df['is_repeated_guest'], hue=df['is_canceled'])

* 大多数预定来源于新客
* 回头客取消预订概率远远低于新客

  建议对新客加大培养，提高对酒店复购，如保持价格最优；在订单支付页面再给一些会员权益；给新客不断派发优惠券，特别是节假日期间或者当地旅游旺季

In [ ]:
# 之前取消预定次数
plt.subplot(121)
plt.plot(df.groupby('previous_cancellations')['is_canceled'].mean(),
         'ro')
plt.xlabel('Previous Cancellations')
# 之前未取消预定次数
plt.subplot(122)
plt.plot(df.groupby('previous_bookings_not_canceled')['is_canceled'].mean(),
         'bo')
plt.ylim(0, 1)
plt.xlabel('Previous Un-Cancellations')

* 先前取消过预定的客户本次预定取消的概率较大，尤其是取消过预定15次以上的客户，基本上不会选择入住，可以计入酒店的“黑名单”
* 先前预定并入住过的客户相对来说信用较好，高入住次数（>20次）客户基本不会取消预订

In [ ]:
sns.countplot(x=df['customer_type'], hue=df['is_canceled'])

* 客户类型主要是过往旅客且订单取消率较高
* Group类型客户最少但几乎没有取消（这里没有搞懂Transient-Party和Group的区别，我理解的是Transient-Party是旅行社，Group是公司团体，如果是这样的话也不难理解，公司团建一般都是惯例且不会因个人原因而轻易取消，而旅行社则是主要取决于个人意愿故取消概率较大）
* 整体看，团体取消概率较个人取消概率较低，可以设定团购优惠，以确保客流量的稳定性。

In [ ]:
# 车位需求统计
sns.countplot(x=df['required_car_parking_spaces'],hue=df['hotel'])

* 多数客户不需要停车位
* 度假酒店客户需要停车位的比例远大于城市酒店。

In [ ]:
# 特殊需求统计
sns.countplot(x=df['total_of_special_requests'],hue=df['is_canceled'])

* 多数客户无特殊需求
* 结果并没有表明客户特殊需求多而取消率高反而有相反趋势，说明酒店有一定特色，在满足客户特殊需求这一方面做得很好，继续保持。

## 订单信息分析

① 订单取消率

In [ ]:
# 计算取消率:
total_cancelations = df["is_canceled"].sum()
rh_cancelations = df.loc[df["hotel"] == "Resort Hotel"]["is_canceled"].sum()
ch_cancelations = df.loc[df["hotel"] == "City Hotel"]["is_canceled"].sum()

rel_cancel = total_cancelations / df.shape[0] * 100
rh_rel_cancel = rh_cancelations / df.loc[df["hotel"] == "Resort Hotel"].shape[0] * 100
ch_rel_cancel = ch_cancelations / df.loc[df["hotel"] == "City Hotel"].shape[0] * 100

print(f"Total bookings canceled: {total_cancelations:,} ({rel_cancel:.0f} %)")
print(f"Resort hotel bookings canceled: {rh_cancelations:,} ({rh_rel_cancel:.0f} %)")
print(f"City hotel bookings canceled: {ch_cancelations:,} ({ch_rel_cancel:.0f} %)")

* 城市酒店预订取消率高于度假酒店，主要是因为城市酒店的主要用户群是商务差旅的用户，往往具有紧急性及未规划性，酒店的预订在未规划及深入了解酒店状态情况下，容易盲目预订、退订，所以退订率高，建议在在渠道平台增加“附近优选”功能，通过输入地址，自动筛选推荐附近城市酒店的入住率高、复住率高、评价高等高品质回馈的城市酒店，一方面能为用户提供更高效便捷的推荐服务，另一方面也促使平台渠道优化服务内容

In [ ]:
# 提前预定时长的分布情况
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.hist(df['lead_time'], bins=50)
plt.xlabel('Lead Time')
plt.ylabel('Number')

# 提前预定时长对取消的影响
plt.subplot(122)
plt.plot(df.groupby('lead_time')['is_canceled'].mean().index,
         df.groupby('lead_time')['is_canceled'].mean(),
         'ro',
         markersize=2)
plt.xlabel('Lead Time')
plt.ylabel('Cancellation rate')

* 客户主要倾向于选择与入住时间相近的时间预定
* 随着预定提前时长的增大，可变因素越多，取消率呈现上升趋势

* 提前预定时间越长，可变因素越多，取消率越高。

In [ ]:
# 生成月份排序列表
ordered_months = [
    "January", "February", "March", "April", "May", "June", "July", "August",
    "September", "October", "November", "December"
]

for hotel in ['City Hotel','Resort Hotel']:
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    df_hotel=df[df.hotel==hotel]
    monthly = df_hotel.groupby('arrival_date_month').size()
    monthly /= 2
    monthly.loc[['July', 'August']] = monthly.loc[['July', 'August']] * 2 / 3
    sns.barplot(x=list(range(1, 13)),y= monthly[ordered_months], ax=ax1)
    ax2.plot(
    range(12), df_hotel.groupby('arrival_date_month')
    ['is_canceled'].mean()[ordered_months].values, 'ro-')
    ax1.set_xlabel('Month')
    ax2.set_ylabel('Cancellation rate')

* 预定量上，城市酒店7/8月出现大幅下滑，同期度假酒店变化较小，整体而言，度假酒店月度客流量变化较小；

* 取消率上，两家酒店冬季取消率相对较低，城市酒店夏季取消率降低，度假酒店却处于高峰。

In [ ]:
# 预定渠道对取消率的影响
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
sns.countplot(
    x=df['distribution_channel'],
    order=df.groupby('distribution_channel')['is_canceled'].mean().index,
    ax=ax1)
ax1.set_xlabel('Distribution Channel')
ax2.plot(df.groupby('distribution_channel')['is_canceled'].mean(), 'ro-')
ax2.set_ylabel('Rate')

1）预定主要来自于旅行社(TA/TO)，个人直接预定(Direct)和团体预定(Group)；

2）旅行社取消预定的概率远大于其他渠道，可能是由于旅行社出于盈利考虑会取消利润较低的订单。

## 其它因素

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(df.corr().abs(), cmap=sns.cm.rocket_r)

* 与取消率相关性较强的五个因素为：
* lead_time:提前预定天数
* total_of_special_requests:客户提出的特殊要求的数量
* required_car_parking_spaces:客户要求的停车位数
* booking_changes:对预订所作的更改/修改的数目
* room_changed:房间类型更改

# Building the Model

In [ ]:
num_features = [
    
    'lead_time', 'arrival_date_week_number', 'arrival_date_day_of_month',
    'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
    'babies', 'is_repeated_guest', 'previous_cancellations',
    'previous_bookings_not_canceled', 'agent', 'adr','required_car_parking_spaces',
    'total_of_special_requests'
]

cat_features = [
    'hotel', 'arrival_date_month', 'meal', 'market_segment',
    'distribution_channel', 'reserved_room_type', 'deposit_type',
    'customer_type'
]

由于数值型特征的单位量纲均不一样，模型拟合时容易偏拟合，所以需要做归一化处理，统一量纲，并保留数据规律

In [ ]:
#数值型特征标准化过程
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

在模型拟合时，模型无法对复杂的标称型特征做拟合，所以需要优先对该类数据进行向量化处理，转化为模型容易辨识的数值型特征

In [ ]:
#标称型特征向量化过程
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
#数值型变量和标称型变量分别做标准化、向量化预处理
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)])   #预处理器

② 划分样本特征和样本结果

In [ ]:
feature = num_features + cat_features
#样本特征
X = df.drop("is_canceled",axis=1)[feature]
#样本结果
y = df["is_canceled"]

③划分训练集和测试集

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

④模型搭建并评估

In [ ]:
base_models = [("DT_model", DecisionTreeClassifier(random_state=42)), # 决策树
               ("RF_model", RandomForestClassifier(random_state=42,n_jobs=-1)),# 随机森林
               ("LR_model", LogisticRegression(random_state=42,n_jobs=-1)),# 逻辑回归
               ("KNN_model",KNeighborsClassifier(n_jobs=-1)),# K近邻
               ("GB_model",GaussianNB())]   # 高斯NB
for name,model in base_models:
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', model)])
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print("%s score: %.3f" % (name,score))

模型选择了决策树、随机森林、逻辑回归、K近邻、高斯NB，分别对模型拟合效果评分，显然“随机森林”的评分0.869，拟合效果更好

⑤模型参数调优

参数调优使用GridSearchCV，参数选择"n_estimators"：决策树的量；"max_depth"：决策树的深度（预剪枝）；"max_features"：选择的最大特征量

In [ ]:
rf = RandomForestClassifier()
#参数选择
param_dict = {"n_estimators":[100,150,200],"max_depth":[3,5,8,10,15],"max_features":["auto","log2"]}
#网络搜索调优器
rf_model = GridSearchCV(rf,param_grid=param_dict,cv=3)

#模型拟合
CLF = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', rf_model)])
CLF.fit(X_train,y_train)

由于处理速度有限，本次只使用一下参数调优

In [ ]:
rf_model = RandomForestClassifier(n_estimators=160,
                               max_features=0.22,
                               min_samples_split=2,
                               n_jobs=-1,
                               random_state=0)
CLF = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', rf_model)])
CLF.fit(X_train, y_train)
CLF.score(X_test, y_test)